<a href="https://colab.research.google.com/github/Godiya-PT/Disater_related_Tweet_Detection/blob/main/Disaster_related_tweet_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Code for Disater related tweets prediction

trial for Disaster related tweet prediction

Import libraries

In [2]:
# !pip install tensorflow_datasets
# !pip install NLP-python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
#from mpl_toolkits import mplot3d
# import spacy
# import re
# import nltk
# import seaborn as sns

# import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorflow import keras
# from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.preprocessing import LabelEncoder
# from collections import defaultdict
# from collections import Counter

# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn import model_selection, naive_bayes
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.decomposition import TruncatedSVD
# from sklearn.preprocessing import MaxAbsScaler
# from scipy import stats
# from sklearn.metrics import make_scorer, roc_auc_score
# from scipy.linalg import svd
# from numpy import diag
# from scipy.sparse import csr_matrix

# from numpy import zeros
# from sklearn import svm

# #from nlp_functions import *

Import dataset

In [6]:
ds_train=pd.read_csv("/train.csv")
ds_test=pd.read_csv("/test.csv")
print("Train and Test data sets are imported successfully")

Train and Test data sets are imported successfully


Checking number of records in train data set and few records from train data set

In [12]:
print("Number of records in Train data set",len(ds_train.index))
ds_train.head()

Number of records in Train data set 7613


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


Distinct keywords in train dataset

In [14]:
#Distinct keywords in train dataset
dist_keyword=ds_train['keyword'].value_counts()
print(dist_keyword)

fatalities               45
armageddon               42
deluge                   42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64


Visualize the keywords

In [26]:

fig = px.scatter(dist_keyword, x=dist_keyword.values, y=dist_keyword.index,size=dist_keyword.values)
fig.show()

Distinct location in train dataset

In [27]:
dist_location=ds_train['location'].value_counts()
#Visualize location
fig = px.scatter(dist_location, y=dist_location.values, x=dist_location.index,size=dist_location.values)
fig.show()

check how many missing values are there 'Location' and 'Keyword' columns

In [8]:
# creating bool series True for NaN values for location 
bool_series_location = pd.isnull(ds_train['location']) 

# filtering data  
# displaying data only with location = NaN  
print(ds_train[bool_series_location])
print("Number of records with missing location",len(ds_train[bool_series_location]))

         id keyword  ...                                               text target
0         1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1         4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2         5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3         6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4         7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1
...     ...     ...  ...                                                ...    ...
7608  10869     NaN  ...  Two giant cranes holding a bridge collapse int...      1
7609  10870     NaN  ...  @aria_ahrary @TheTawniest The out of control w...      1
7610  10871     NaN  ...  M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...      1
7611  10872     NaN  ...  Police investigating after an e-bike collided ...      1
7612  10873     NaN  ...  The Latest: More Homes Razed by Northern Calif...      1

[25

Creating bool series True for NaN values for key word

In [9]:
bool_series_keyword = pd.isnull(ds_train['keyword']) 
# filtering data  
# displaying data only with Keywords = NaN  
ds_train[bool_series_keyword]
print("Number of records with missing keywords",len(ds_train[bool_series_keyword]))

Number of records with missing keywords 61


Calculate percentage of missing keywords

In [10]:
print('{}% of Kewords are missing from Total Number of Records'.format((len(ds_train[bool_series_keyword])/len(ds_train.index))*100))

0.8012610009194797% of Kewords are missing from Total Number of Records


From Data description and above analysis we can conclude :

Locations are not automatically generated, they are user inputs and that's why data is not clean and there are too many incoorect and missing values.We can skip the 'location' column from our feature list

We can consider the 'keyword' column as a feature because there are a lot of unique keywords and missing values are very insignificant (< 1 percentage)

In [11]:
#dropping unwanted column 'location'
ds_train=ds_train.drop(['location'],axis=1)
ds_train.head()

,id,keyword,text,target
0,1,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
#dropping missing 'keyword' records from train data set
ds_train=ds_train.drop(ds_train[bool_series_keyword].index,axis=0)
#Resetting the index after droping the missing records
ds_train=ds_train.reset_index(drop=True)
print("Number of records after removing missing keywords",len(ds_train))
ds_train.head()

Number of records after removing missing keywords 7552


,id,keyword,text,target
0,48,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
1,49,ablaze,We always try to bring the heavy. #metal #RT h...,0
2,50,ablaze,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
3,52,ablaze,Crying out for more! Set me ablaze,0
4,53,ablaze,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
